In [72]:
import pandas as pd
import os
import sys
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Add project root to path (for Jupyter notebooks)
# Get the current directory and navigate to project root
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
sys.path.insert(0, project_root)

from src.config import Config
import src.eda as eda
import src.present_value as present_value
from src.ml_linear_dependency import train_and_calculate_metrics
from src.ml_geotecnia import train_geotecnia_model, prepare_geotecnia_data
from src.ml_bridges_structures import train_brindges_structures_model
from src.ml_tunnels import train_tunnel_model
from src.ml_paisajismo import train_paisajismo_model, prepare_paisajismo_data
from src.ml_cantidades_socioeconomica import train_cantidades_model

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
## FROM DATABASE
pv = present_value.PresentValue()
anual_increment = pv.fetch_incremento_from_database()

fase = "III"
preproccesing = eda.EDA()
df_raw = preproccesing.assemble_projects_from_database(fase)
df_vp = preproccesing.create_dataset(pv.present_value_costs, fase=fase)

In [75]:
predictors = ['LONGITUD KM']
hue_name = 'ALCANCE'

targets = []

if fase == 'II':
    targets = ['1 - TRANSPORTE', '2 - TRAZADO Y TOPOGRAFIA (incluye subcomponentes)', 
                      '3 - GEOLOGÍA (incluye subcomponentes)', '8 - PAVIMENTO', '9 - PREDIAL', '10 - AMBIENTAL Y SOCIAL',
                      '11 - COSTOS Y PRESUPUESTOS', '12 - SOCIOECONÓMICA', '13 - DIRECCIÓN Y COORDINACIÓN']
elif fase == 'III': 
    targets = ['1 - TRANSPORTE', '2.1 - INFORMACIÓN GEOGRÁFICA','2.2 - TRAZADO Y DISEÑO GEOMÉTRICO', 
                      '2.3 - SEGURIDAD VIAL', '2.4 - SISTEMAS INTELIGENTES', '5 - TALUDES', '6 - PAVIMENTO',
                      '7 - SOCAVACIÓN', '11 - PREDIAL', '12 - IMPACTO AMBIENTAL', '15 - OTROS - MANEJO DE REDES']

df = df_vp[['LONGITUD KM', 'ALCANCE']].join(df_vp.loc[:, '1 - TRANSPORTE':'16 - DIRECCIÓN Y COORDINACIÓN'])
results = train_and_calculate_metrics(df, targets, predictors, hue_name)    

1 - TRANSPORTE
  → Warning: Only 2 samples. Skipping outlier detection.
{'R2': -3.0, 'MAE': 4643975.471, 'RMSLE': 0.236, 'MAPE%': 23.791}
2.1 - INFORMACIÓN GEOGRÁFICA
  → Removed 1/10 outliers (10.0%) using ensemble
{'R2': 1.0, 'MAE': 199650.365, 'RMSLE': 0.096, 'MAPE%': 5.977}
2.2 - TRAZADO Y DISEÑO GEOMÉTRICO
  → Removed 5/50 outliers (10.0%) using ensemble
{'R2': 0.826, 'MAE': 2270169.486, 'RMSLE': 0.384, 'MAPE%': 22.302}
2.3 - SEGURIDAD VIAL
  → Removed 4/49 outliers (8.2%) using ensemble
{'R2': 0.792, 'MAE': 2511792.659, 'RMSLE': 0.494, 'MAPE%': 35.996}
2.4 - SISTEMAS INTELIGENTES
  → Removed 5/42 outliers (11.9%) using ensemble
{'R2': 0.851, 'MAE': 409025.912, 'RMSLE': 0.127, 'MAPE%': 4.59}
5 - TALUDES
  → Removed 7/44 outliers (15.9%) using ensemble
{'R2': 0.55, 'MAE': 4610935.437, 'RMSLE': 0.617, 'MAPE%': 21.407}
6 - PAVIMENTO
  → Removed 4/34 outliers (11.8%) using ensemble
{'R2': 0.241, 'MAE': 4531609.341, 'RMSLE': 0.652, 'MAPE%': 30.96}
7 - SOCAVACIÓN
  → Removed 5/50 outlie

In [76]:
predictors = ["2.2 - TRAZADO Y DISEÑO GEOMÉTRICO", "5 - TALUDES", "7 - SOCAVACIÓN"]
targets = ['16 - DIRECCIÓN Y COORDINACIÓN']
hue_name = ''
results['16 - DIRECCIÓN Y COORDINACIÓN'] = train_and_calculate_metrics(df, targets, predictors)['16 - DIRECCIÓN Y COORDINACIÓN']  

16 - DIRECCIÓN Y COORDINACIÓN
  → Removed 7/51 outliers (13.7%) using ensemble
{'R2': 0.95, 'MAE': 1516804.865, 'RMSLE': 0.137, 'MAPE%': 6.552}


In [77]:
df = prepare_geotecnia_data(df_vp)
predictors = ["2.2 - TRAZADO Y DISEÑO GEOMÉTRICO", "5 - TALUDES", "7 - SOCAVACIÓN"]
target = "3 - GEOLOGÍA"
results[target] = train_geotecnia_model(df, predictors, target)


  → Warning: Only 6 samples. Skipping outlier detection.


In [78]:
predictors = ['PUENTES VEHICULARES M2']
target= '4 - SUELOS'
results[target] = train_brindges_structures_model(df_vp, target, predictors, exclude_codes=['0654801'], use_log_transform=True)


  → Warning: Only 7 samples. Skipping outlier detection.
{'Model': 'Model', 'R²': 0.931, 'MAE': 58727741.626, 'RMSE': 75610324.783, 'MAPE (%)': 39.004, 'Median AE': 35176571.996, 'Max Error': 132427513.227}


In [79]:
predictors = ['PUENTES VEHICULARES UND']
target= '8 - ESTRUCTURAS'
results[target] = train_brindges_structures_model(df_vp, target, predictors, exclude_codes=['0654801'], use_log_transform=False)


  → Warning: Only 6 samples. Skipping outlier detection.
{'Model': 'Model', 'R²': 0.911, 'MAE': 106072066.135, 'RMSE': 124728195.953, 'MAPE (%)': 36.056, 'Median AE': 86741275.295, 'Max Error': 233861419.752}


In [80]:
target = '9 - TÚNELES'
predictors = ['TUNELES UND', 'TUNELES KM']
results[target] = train_tunnel_model(df_vp, predictors, target)


R² = 0.8340 | MAPE = 9.76%


In [81]:
df = prepare_paisajismo_data(df_vp)
predictors = ['PUENTES PEATONALES UND'] 
target = '10 - URBANISMO Y PAISAJISMO'
results[target] = train_paisajismo_model(df, predictors, target)

  → Warning: Only 2 samples. Skipping outlier detection.
{'Model': 'Linear Regression', 'R²': 1.0, 'MAE': 0.0, 'RMSE': 0.0, 'MAPE (%)': 0.0, 'Median AE': 0.0, 'Max Error': 0.0}


In [82]:
from src.ml_cantidades_socioeconomica import train_cantidades_model

predictors = ['PUENTES VEHICULARES UND', 'PUENTES VEHICULARES M2', 'PUENTES PEATONALES UND']
target = '13 - CANTIDADES'
results[target] = train_cantidades_model(df_vp, predictors, target, log_transform='none')

  → Warning: Only 5 samples. Skipping outlier detection.
{'Model': 'Ridge', 'R²': 0.995, 'MAE': 1021210.421, 'RMSE': 1153118.706, 'MAPE (%)': 9.334, 'Median AE': 717706.14, 'Max Error': 1739110.278}


In [83]:
results.keys()

dict_keys(['1 - TRANSPORTE', '2.1 - INFORMACIÓN GEOGRÁFICA', '2.2 - TRAZADO Y DISEÑO GEOMÉTRICO', '2.3 - SEGURIDAD VIAL', '2.4 - SISTEMAS INTELIGENTES', '5 - TALUDES', '6 - PAVIMENTO', '7 - SOCAVACIÓN', '11 - PREDIAL', '12 - IMPACTO AMBIENTAL', '15 - OTROS - MANEJO DE REDES', '16 - DIRECCIÓN Y COORDINACIÓN', '3 - GEOLOGÍA', '4 - SUELOS', '8 - ESTRUCTURAS', '9 - TÚNELES', '10 - URBANISMO Y PAISAJISMO', '13 - CANTIDADES'])